# Lab 06 Softmax Classification
* Softmax를 사용하여분류를 진행합니다.

### 기본 Library 선언 및 Tensorflow 버전 확인

In [2]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
tf.set_random_seed(777)  # for reproducibility
tfe = tf.contrib.eager

In [8]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [9]:
#dataset을 선언합니다.
# dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
# dataset = dataset.repeat().batch(2)

nb_classes = 3 #class의 개수입니다.

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [10]:
#Weight and bias setting
W = tfe.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tfe.Variable(tf.random_normal([nb_classes]), name='bias')
variables = [W, b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.7706481 ,  0.37335402, -0.05576323],
       [ 0.00358377, -0.5898363 ,  1.5702795 ],
       [ 0.2460895 , -0.09918973,  1.4418385 ],
       [ 0.3200988 ,  0.526784  , -0.7703731 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.3080608 , -0.13253094,  0.5513761 ], dtype=float32)>


In [11]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def softmax_regression(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(softmax_regression(x_data))

tf.Tensor(
[[1.36571955e-02 7.90162385e-03 9.78441238e-01]
 [3.92597765e-02 1.70347411e-02 9.43705440e-01]
 [3.80385160e-01 1.67723209e-01 4.51891690e-01]
 [3.23390663e-01 5.90759777e-02 6.17533386e-01]
 [3.62997412e-06 6.20727292e-08 9.99996305e-01]
 [2.62520462e-02 1.07279727e-02 9.63019967e-01]
 [1.56525111e-05 4.21802781e-07 9.99983907e-01]
 [2.94076904e-06 3.81133276e-08 9.99997020e-01]], shape=(8, 3), dtype=float32)


In [17]:
# Softmax onehot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(softmax_regression(sample_db))

tf.Tensor([[0.9302204  0.06200533 0.00777428]], shape=(1, 3), dtype=float32)


In [29]:
def cost_fn(X, Y):
    logits = softmax_regression(X)
    cost = -tf.reduce_sum(Y * tf.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(6.07932, shape=(), dtype=float32)


In [30]:
def cost_fn(X, Y):
    logits = softmax_regression(X)
    cost = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y)
    cost_mean = tf.reduce_mean(cost)
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(1.2151728, shape=(), dtype=float32)


In [32]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x # x^2
dy_dx = g.gradient(y, x) # Will compute to 6.0
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [36]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)

        return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: id=610, shape=(4, 3), dtype=float32, numpy=
array([[ 0.06991257, -0.00614021, -0.06377237],
       [ 0.02443989, -0.00085829, -0.02358155],
       [ 0.06863485, -0.01986565, -0.04876918],
       [ 0.08595259, -0.01490254, -0.07105   ]], dtype=float32)>, <tf.Tensor: id=608, shape=(3,), dtype=float32, numpy=array([ 0.02441996,  0.00028941, -0.02470937], dtype=float32)>]


In [95]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, loss_fn(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 0.159370
Loss at epoch 100: 0.153950
Loss at epoch 200: 0.148822
Loss at epoch 300: 0.144014
Loss at epoch 400: 0.139495
Loss at epoch 500: 0.135241
Loss at epoch 600: 0.131231
Loss at epoch 700: 0.127443
Loss at epoch 800: 0.123861
Loss at epoch 900: 0.120468
Loss at epoch 1000: 0.117251
Loss at epoch 1100: 0.114196
Loss at epoch 1200: 0.111291
Loss at epoch 1300: 0.108526
Loss at epoch 1400: 0.105891
Loss at epoch 1500: 0.103378
Loss at epoch 1600: 0.100977
Loss at epoch 1700: 0.098683
Loss at epoch 1800: 0.096488
Loss at epoch 1900: 0.094386
Loss at epoch 2000: 0.092371


# Class Version

In [40]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tfe.Variable(tf.random_normal([4, nb_classes]), name='weight')
        self.b = tfe.Variable(tf.random_normal([nb_classes]), name='bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(logits), axis=1))
        
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)
            
            return grads
    
def fit(self, X, Y, epochs=2000, verbose=500):
    optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.1)

    for i in range(epochs):
        grads = self.grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, self.variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, self.loss_fn(X, Y).numpy()))

In [41]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
model = softmax_classifer(nb_classes)

In [ ]:
model.fit(x_data, y_data)